# 0. Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("data/sonar_dataset.csv", header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


# 1. Preprocessing

## 1.1 Check for null value

In [18]:
# Count the number of null values in each row
null_counts = df.isna().sum(axis=1)

null_rows = []

for i in null_counts.index:
    if null_counts.values[i] != 0:
        null_rows.append(i)

null_rows

[]

## 1.2 Check for imbalance data

In [21]:
df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

## 1.3 Preparing X and y

In [23]:
X = df.drop(60, axis="columns")
y = df[60]

In [24]:
X.shape

(208, 60)

In [25]:
y.shape

(208,)

In [28]:
y = pd.get_dummies(y, drop_first=True)
y.value_counts()

R
0    111
1     97
dtype: int64

# 2. Model

## 2.1 Original model

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

In [30]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5256
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6760 - accuracy: 0.6154
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6596 - accuracy: 0.6474
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6381 - accuracy: 0.6795
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6063 - accuracy: 0.7628
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5738 - accuracy: 0.7756
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5190 - accuracy: 0.8205
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4816 - accuracy: 0.7949
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.8269
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4008 - accuracy: 0.8141
Epoch 11/

In [31]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 1.2385 - accuracy: 0.7500


[1.238544225692749, 0.75]

In [34]:
from sklearn.metrics import confusion_matrix , classification_report

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        28
           1       0.70      0.79      0.75        24

    accuracy                           0.75        52
   macro avg       0.75      0.75      0.75        52
weighted avg       0.76      0.75      0.75        52



## 2.2 Dropout Model

In [35]:
model2 = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.7618 - accuracy: 0.4359
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7054 - accuracy: 0.4551
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6994 - accuracy: 0.5641
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6994 - accuracy: 0.5000
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6988 - accuracy: 0.5064
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6986 - accuracy: 0.5513
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6948 - accuracy: 0.5000
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6912 - accuracy: 0.5192
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6783 - accuracy: 0.5577
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6702 - accuracy: 0.5833
Epoch 11/

In [36]:
model2.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.5701 - accuracy: 0.7115


[0.5701169371604919, 0.7115384340286255]

In [37]:
from sklearn.metrics import confusion_matrix , classification_report

y_pred = model2.predict(X_test)
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

2/2 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.72      0.75      0.74        28
           1       0.70      0.67      0.68        24

    accuracy                           0.71        52
   macro avg       0.71      0.71      0.71        52
weighted avg       0.71      0.71      0.71        52

